In [1]:
#@title Huggingface Login
#@markdown huggingface weight 를 이용하고 싶다면 로그인 필수
from google.colab import userdata
import os

os.environ['HF_WRITE_TOKEN'] = userdata.get('HF_WRITE_TOKEN')

!huggingface-cli login --add-to-git-credential --token $HF_WRITE_TOKEN


Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
#@markdown install with openvino
%%sh
# apt-get update  -y
# apt-get install -y gcc-12 g++-12
# update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-12 10 --slave /usr/bin/g++ g++ /usr/bin/g++-12
# pip install --upgrade pip
# pip install wheel packaging ninja "setuptools>=49.4.0" numpy
git clone https://github.com/vllm-project/vllm.git
cd vllm && pip install -r requirements-build.txt --extra-index-url https://download.pytorch.org/whl/cpu
pip install gguf
PIP_EXTRA_INDEX_URL="https://download.pytorch.org/whl/cpu https://storage.openvinotoolkit.org/simple/wheels/pre-release" && \
    VLLM_TARGET_DEVICE=openvino python -m pip install -v .

Output hidden; open in https://colab.research.google.com to view.

In [6]:
#@markdown install with cpu
# %%sh
# apt-get update  -y
# apt-get install -y gcc-12 g++-12
# update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-12 10 --slave /usr/bin/g++ g++ /usr/bin/g++-12
# pip install --upgrade pip
# pip install wheel packaging ninja "setuptools>=49.4.0" numpy
# pip install pynvml
# git clone https://github.com/vllm-project/vllm.git
# cd vllm && pip install -U -q -v -r requirements-cpu.txt --extra-index-url https://download.pytorch.org/whl/cpu
# VLLM_TARGET_DEVICE=cpu python setup.py install

Reading package lists...
Building dependency tree...
Reading state information...
gcc-12 is already the newest version (12.3.0-1ubuntu1~22.04).
g++-12 is already the newest version (12.3.0-1ubuntu1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu
  Using cached tokenizers-0.20.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
running install
running bdist_egg
running egg_info
writing vllm.egg-info/PKG-INFO
writing dependency_links to vllm.egg-info/dependency_links.txt
writing entry points to vllm.egg-info/entry_points.txt
writing requirements to vllm.egg-info/requires.txt
writing top-level names to vllm.egg-info/top_level.txt
reading manifest file 'vllm.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE'
writing manifest file 'vllm.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
runnin

fatal: destination path 'vllm' already exists and is not an empty directory.
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/install

In [3]:
#@markdown colab installation
!pip install pynvml
!VLLM_TARGET_DEVICE=cpu pip install -U vllm ray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 MB 37.9 MB/s eta 0:00:00


In [4]:
from vllm import LLM, SamplingParams
import os

os.environ["VLLM_CPU_KVCACHE_SPACE"] = "10"
os.environ["VLLM_CPU_OMP_THREADS_BIND"] = "0-27"

if "model" in locals():
    del model

model_id = "microsoft/Phi-3.5-mini-instruct"
model_id = "KISTI-KONI/KONI-Llama3-8B-Instruct-20240729"
model_id = "Gunulhona/Llama-Merge-Small"
model_id = "akjindal53244/Llama-3.1-Storm-8B"

model  = LLM(
    model=model_id,
    max_model_len=4096,
    trust_remote_code=True,
    quantization="gguf",
    dtype="bfloat16",
    distributed_executor_backend="ray",
    )

INFO 08-27 05:12:41 importing.py:10] Triton not installed; certain GPU-related functions will not be available.
WARNING 08-27 05:12:41 _custom_ops.py:17] Failed to import from vllm._C with ModuleNotFoundError("No module named 'vllm._C'")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/910 [00:00<?, ?B/s]

WARNING 08-27 05:12:50 config.py:320] gguf quantization is not fully optimized yet. The speed can be slower than non-quantized models.
WARNING 08-27 05:12:50 config.py:344] Async output processing is only supported for CUDA. Disabling it for other platforms.
INFO 08-27 05:12:50 llm_engine.py:198] Initializing an LLM engine (v0.5.5) with config: model='akjindal53244/Llama-3.1-Storm-8B', speculative_config=None, tokenizer='akjindal53244/Llama-3.1-Storm-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=gguf, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cpu, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(ot

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Using framework PyTorch: 2.4.0+cu121
Overriding 1 configuration item(s)
	- use_cache -> True
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/usr/local/lib/python3.10/dist-packages/optimum/exporters/openvino/model_patcher.py:479: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if sequence_length != 1:


INFO 08-27 05:27:44 openvino_executor.py:73] # CPU blocks: 1024


In [20]:
import gc
import os
from typing import List
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

def chat_format(prompt:List[dict])->str:
    tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
    try:
        check = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
        print("tokenizer has format")
    except:
        tokenizer.bos_token = "<|begin_of_text|>"
        tokenizer.chat_template= "{{- bos_token }}\n{%- if custom_tools is defined %}\n    {%- set tools = custom_tools %}\n{%- endif %}\n{%- if not tools_in_user_message is defined %}\n    {%- set tools_in_user_message = true %}\n{%- endif %}\n{%- if not date_string is defined %}\n    {%- set date_string = \"26 Jul 2024\" %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n\n{#- This block extracts the system message, so we can slot it into the right place. #}\n{%- if messages[0]['role'] == 'system' %}\n    {%- set system_message = messages[0]['content']|trim %}\n    {%- set messages = messages[1:] %}\n{%- else %}\n    {%- set system_message = \"\" %}\n{%- endif %}\n\n{#- System message + builtin tools #}\n{{- \"<|start_header_id|>system<|end_header_id|>\\n\\n\" }}\n{%- if builtin_tools is defined or tools is not none %}\n    {{- \"Environment: ipython\\n\" }}\n{%- endif %}\n{%- if builtin_tools is defined %}\n    {{- \"Tools: \" + builtin_tools | reject('equalto', 'code_interpreter') | join(\", \") + \"\\n\\n\"}}\n{%- endif %}\n{{- \"Cutting Knowledge Date: December 2023\\n\" }}\n{{- \"Today Date: \" + date_string + \"\\n\\n\" }}\n{%- if tools is not none and not tools_in_user_message %}\n    {{- \"You have access to the following functions. To call a function, please respond with JSON for a function call.\" }}\n    {{- 'Respond in the format {\"name\": function name, \"parameters\": dictionary of argument name and its value}.' }}\n    {{- \"Do not use variables.\\n\\n\" }}\n    {%- for t in tools %}\n        {{- t | tojson(indent=4) }}\n        {{- \"\\n\\n\" }}\n    {%- endfor %}\n{%- endif %}\n{{- system_message }}\n{{- \"<|eot_id|>\" }}\n\n{#- Custom tools are passed in a user message with some extra guidance #}\n{%- if tools_in_user_message and not tools is none %}\n    {#- Extract the first user message so we can plug it in here #}\n    {%- if messages | length != 0 %}\n        {%- set first_user_message = messages[0]['content']|trim %}\n        {%- set messages = messages[1:] %}\n    {%- else %}\n        {{- raise_exception(\"Cannot put tools in the first user message when there's no first user message!\") }}\n{%- endif %}\n    {{- '<|start_header_id|>user<|end_header_id|>\\n\\n' -}}\n    {{- \"Given the following functions, please respond with a JSON for a function call \" }}\n    {{- \"with its proper arguments that best answers the given prompt.\\n\\n\" }}\n    {{- 'Respond in the format {\"name\": function name, \"parameters\": dictionary of argument name and its value}.' }}\n    {{- \"Do not use variables.\\n\\n\" }}\n    {%- for t in tools %}\n        {{- t | tojson(indent=4) }}\n        {{- \"\\n\\n\" }}\n    {%- endfor %}\n    {{- first_user_message + \"<|eot_id|>\"}}\n{%- endif %}\n\n{%- for message in messages %}\n    {%- if not (message.role == 'ipython' or message.role == 'tool' or 'tool_calls' in message) %}\n        {{- '<|start_header_id|>' + message['role'] + '<|end_header_id|>\\n\\n'+ message['content'] | trim + '<|eot_id|>' }}\n    {%- elif 'tool_calls' in message %}\n        {%- if not message.tool_calls|length == 1 %}\n            {{- raise_exception(\"This model only supports single tool-calls at once!\") }}\n        {%- endif %}\n        {%- set tool_call = message.tool_calls[0].function %}\n        {%- if builtin_tools is defined and tool_call.name in builtin_tools %}\n            {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' -}}\n            {{- \"<|python_tag|>\" + tool_call.name + \".call(\" }}\n            {%- for arg_name, arg_val in tool_call.arguments | items %}\n                {{- arg_name + '=\"' + arg_val + '\"' }}\n                {%- if not loop.last %}\n                    {{- \", \" }}\n                {%- endif %}\n                {%- endfor %}\n            {{- \")\" }}\n        {%- else  %}\n            {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' -}}\n            {{- '{\"name\": \"' + tool_call.name + '\", ' }}\n            {{- '\"parameters\": ' }}\n            {{- tool_call.arguments | tojson }}\n            {{- \"}\" }}\n        {%- endif %}\n        {%- if builtin_tools is defined %}\n            {#- This means we're in ipython mode #}\n            {{- \"<|eom_id|>\" }}\n        {%- else %}\n            {{- \"<|eot_id|>\" }}\n        {%- endif %}\n    {%- elif message.role == \"tool\" or message.role == \"ipython\" %}\n        {{- \"<|start_header_id|>ipython<|end_header_id|>\\n\\n\" }}\n        {%- if message.content is mapping or message.content is iterable %}\n            {{- message.content | tojson }}\n        {%- else %}\n            {{- message.content }}\n        {%- endif %}\n        {{- \"<|eot_id|>\" }}\n    {%- endif %}\n{%- endfor %}\n{%- if add_generation_prompt %}\n    {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' }}\n{%- endif %}\n"
        tokenizer.clean_up_tokenization_spaces =True
        # tokenizer.eos_token = "<|eot_id|>"
        print("tokenizer doesn't have format")
    finally:
        prompt = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
    return prompt

os.environ["VLLM_USE_MODELSCOPE"] = "True"

prompt = """
---
[대화]
참석자_1: 안녕하세요, 선생님.
참석자_2: 안녕하세요, 의사 선생님.
참석자_1: 마흔 넷이시죠?
참석자_2: 네, 선생님.
참석자_1: 좋아요, 오늘은 무슨 문제가 있는 것 같나요?
참석자_2: 의사 선생님, 한동안 허리 통증이 있었습니다.
참석자_1: 통증이 다리로 내려가나요?
참석자_2: 네, 오른쪽 허벅지에도 통증이 있습니다.
참석자_1: 이 통증과 관련된 부상이 있습니까?
참석자_2: 네, 1994년에 사고가 있었습니다.
참석자_1: 최초 부상 당시의 서류나 의료 기록이 있습니까?
참석자_2: 아니요, 오늘은 없습니다.
참석자_1: 직업이 어떻게 되십니까?
참석자_2: 지금은 타코벨에서 일합니다. 산재 보험 청구가 열려 있습니다.
참석자_1: 거기서 일하다가 통증이 재발했죠?
참석자_2: 네, 맞습니다.
참석자_1: 마지막으로 이곳에서 진료를 받은 것이 언제였는지 기억하십니까?
참석자_2: 음, 네, 4월 12일 2005년이었습니다.
참석자_1: 10이 상상할 수 있는 최악의 통증이라면, 마지막 방문 시 통증은 10점 만점에 어느 정도였습니까?
참석자_2: 음, 10점 만점에 8점 정도였어요.
참석자_1: 이 통증 때문에 약을 복용하셨나요?
참석자_2: 음, 지난번 방문했을 때 메드롤 도스팩을 처방받았습니다.
참석자_1: 도세팍에 통증이 어떻게 반응했나요?
참석자_2: 통증이 10점 만점에 4~5점 정도로 줄었습니다.
참석자_1: 통증이 있는 곳을 가리켜 주시겠습니까?
참석자_2: 네, 바로 여기입니다.
참석자_1: 여기 이 밴드요?
참석자_2: 네, 바로 그 자리입니다.
참석자_1: 좋아요, 여기는 요추 4번과 천골 사이입니다. 오른쪽 다리 통증을 어떻게 설명하시겠습니까?
참석자_2: 지금은 간헐적이고 미미하며 항상 있는 것은 아닙니다.
참석자_1: 허리 수술을 받은 적이 있습니까?
참석자_2: 음, 네, 1990년에 한 번, 1994년에 한 번 두 번 척추 절제술을 받았습니다. 잠깐만요, 그 사이에 디스크 절제술도 받았어요.
참석자_1: 어디에 초점이 맞춰졌는지 아십니까?
참석자_2: L 4 L 5번이었습니다.
참석자_1: 허리에 대한 영상 촬영은 하셨나요?
참석자_2: 네, 10월 18일 2004년에 MRI를 찍었습니다. 여기 보고서가 있습니다.
참석자_1: 좋아요, 이것은 다단계 퇴행성 변화를 보여 주며, L 2 L 3, L 3 L 4, L 5 S1에서 신경 침범이 없는 다단계 퇴행성 변화를 보여 주며, 이는 양호합니다.
참석자_2: 그게 무슨 뜻인가요, 의사 선생님?
참석자_1: 요약하자면, 허리에 상당한 양의 관절염이 있다는 뜻입니다.
참석자_2: 네, M R 골수 조영술도 받았는데 여기 보고서가 있습니다.
참석자_1: 좋아요, 요추 3번에서 심한 척추관 협착증이 보이지만 인공물일 수도 있습니다.
참석자_2: 그게 무슨 뜻인가요?
참석자_1: 이 소견은 잘못된 해석일 수 있습니다.
 ---
[리포트]
"""

DEFAULT_SUMMARY_SYSTEM_PROMPT = '''
You will be provided with a conversation transcript.
Your task is to summarize the content by extracting the key information and presenting it concisely.
The summary should capture the most important points discussed in the conversation, focusing on essential details.
When summarizing, highlight the most important keywords that capture the essence of the conversation.

Follow these steps to complete the task:
    Step 1: Comprehension - Carefully read the entire conversation transcript from start to finish.
    Step 2: Identification - Identify the main ideas and critical pieces of information, focusing on the core elements of the discussion.
    Step 3: Condensation - Write a summary that condenses the content into 3 sentences. Ensure that the summary is clear, coherent, and accurately reflects the core of the conversation without including unnecessary details.

Finally, present the summary in the following format:
---
[대화] (this is the given dialogue to summarize)
참석자_1: ...
참석자_2: ...
참석자_1: ...
...
중략
...
참석자_1: ...
참석자_2: ... (the dialouge must not be generated)
---
[리포트] (this is the summary to be generated)
환자 정보: (환자의 나이대와 성별, 언급되지 않은 경우 "식별되지 않음"으로 작성)
가족력: (가족력, 언급되지 않은 경우 "식별되지 않음"으로 작성)
내원사유: (내원한 사유, 언급되지 않은 경우 "식별되지 않음"으로 작성)
내원이력: (최근 내원 이력, 언급되지 않은 경우 "식별되지 않음"으로 작성)
재진일정: (재진일정, 언급되지 않은 경우 "식별되지 않음"으로 작성)
정리:
* (내원 이유에 대한 개조식 요약, 언급되지 않은 경우 "식별되지 않음"으로 작성)
* (진단 내용에 대한 개조식 요약, 언급되지 않은 경우 "식별되지 않음"으로 작성)
* (치료 방법에 대한 개조식 요약, 언급되지 않은 경우 "식별되지 않음"으로 작성)
---
'''

template = chat_format([
    {"role": "system",
     "content": DEFAULT_SUMMARY_SYSTEM_PROMPT},
    {"role": "user",
     "content": prompt}
    ])
output_list = model.generate(
    prompts=[template] * 4,
    sampling_params=SamplingParams(
        repetition_penalty=1.0,
        frequency_penalty=1.0,
        presence_penalty=1.0,
        temperature=0.5,
        top_p=0.9,
        max_tokens=500,
    ))
for result in output_list:
    print(f'''
    --- Result ---

    {result.outputs[0].text}

    --- end ---
    ''')

gc.collect()


tokenizer has format


Processed prompts: 100%|██████████| 4/4 [09:29<00:00, 142.43s/it, est. speed input: 10.24 toks/s, output: 1.62 toks/s]



    --- Result ---

    환자 정보: 50대 남성
가족력: 식별되지 않음
내원사유: 허리 통증
내원이력: 1990년, 1994년에 척추 절제술을 받았고, 디스크 절제술도 받은 경력이 있다. 최근 내원은 지난번 방문에서 메드롤 도스팩을 처방받아 사용한 적이 있으며, 통증이 개선되었다.
재진일정: 식별되지 않음
정리:
* 환자는 허리통증으로 내원하여 수술 기록과 관련된 부상을 언급했다. 그러나 의료기록이 존재하지 않는다.
* 환자는 직업으로 타코벨에서 일하고 있으며 산재 보험 청구가 진행 중이다.
* MRI 결과는 다단계 퇴행성 변화를 보여주며 양호하지만 L3-L4의 신경 침범은 없는 것으로 나타났다. 이는 허리에 상당한 양의 관절염이 있다는 것을 의미한다.
* M R 골수 조영술 결과 요추 3번에서 심한 청추관 협착증이 있지만 인공물일 수도 있는 소견이다.

    --- end ---
    

    --- Result ---

    환자 정보: 50대 남성
가족력: 식별되지 않음
내원사유: 허리 통증
내원이력: 1990년, 1994년에 척추 절제술을 받았으며 디스크 절제술도 한 번 받음. 지난번 방문은 2005년 4월에 있었고, MRI는 2004년 10월에 받음.
재진일정: 식별되지 않음
정리:
* 환자는 마흔 넷 살의 남자로 허리통증으로 내원하였습니다. 그는 과거 사고를 당해 부상을 입었으며, 이전에 척추절제술과 디스크절제수를 받은 경험이 있습니다.
* 환자는 최근 직장에서 일하다가 통증이 재발했습니다. 그는 산재 보험 청구를 신청하고 있었습니다.
* 진단 결과, 환자의 허리는 다단계 퇴행성 변화를 보여주며 상당한 양의 관절염이 있다고 판명났습니다. 또한 요추3번에서 심한 청추관 협착증이 발견되었습니다; 그러나 이는 인공물일 수도 있다는 잠정적인 해석입니다.

    --- end ---
    

    --- Result ---

    환자 정보: 50대 남성
가족력: 식별되지 않음
내원사유: 허리 통증
내원이력: 최근 내원 이

0

 Blue whale. The blue whale is not only the largest animal but also the largest
